# Загрузка изображений
Создадим поток загрузки изображений для обучения/проверки из директорий на жестком диске.

Используем обученную нейросеть VGG без последнего слоя для классификации изображений по классам.

При загрузке изменим размеры изображений до 32x32 (5 слоев подвыборки) и приведем их к полноцветным. Обучим последние слои нейросети на этом наборе и проверим точность.

Данные: https://video.ittensive.com/machine-vision/symbols.zip

![VGG](vgg16-neural-network-1.jpg)

### Подключение библиотек

In [1]:
import keras as K
from sklearn.metrics import classification_report

Using TensorFlow backend.


### Подготовка генераторов изображений
Каждый класс изображений находится в своей папке внутри директории symbols:

0-9 - цифры, 10 - A, 11 - B, 12 - C, 13 - E, 14 - H, 15 - K, 16 - M, 17 - P, 18 - T, 19 - X, 20 - Y

In [2]:
train_generator = K.preprocessing.image.ImageDataGenerator().flow_from_directory(
        'symbols', target_size=(32, 32), batch_size=32, shuffle=True)
validation_generator = K.preprocessing.image.ImageDataGenerator().flow_from_directory(
        'symbols', target_size=(32, 32), batch_size=32, shuffle=True)

Found 15650 images belonging to 21 classes.
Found 15650 images belonging to 21 classes.


### Загрузка обученной модели
Без последнего слоя - его и будем обучать

In [3]:
base_model = K.applications.vgg16.VGG16(weights='imagenet', include_top=False)

### Фиксация всех текущих весов модели
И добавление последнего слоя для классификации - всего 21 класс, цифры + буквы

In [4]:
for layer in base_model.layers:
    layer.trainable = False
x = base_model.output
x = K.layers.GlobalAveragePooling2D()(x)
x = K.layers.Dense(4096, activation='relu')(x)
x = K.layers.Dense(21, activation='softmax')(x)
model = K.models.Model(inputs=base_model.input, outputs=x)
model.compile(optimizer=K.optimizers.Adamax(learning_rate=1e-2),
              loss=K.losses.categorical_crossentropy)

### Обучение нейросети

In [6]:
model.fit_generator(
        train_generator,
        steps_per_epoch=480,
        epochs=20,
        validation_data=validation_generator,
        validation_steps=96)

Epoch 1/100
480/480 [==============================] - 92s 192ms/step - loss: 0.3260 - val_loss: 0.0097
Epoch 2/100
480/480 [==============================] - 86s 179ms/step - loss: 0.1193 - val_loss: 0.1251
Epoch 3/100
480/480 [==============================] - 81s 168ms/step - loss: 0.0853 - val_loss: 0.0018
Epoch 4/100
480/480 [==============================] - 77s 161ms/step - loss: 0.0771 - val_loss: 0.0096
Epoch 5/100
480/480 [==============================] - 76s 159ms/step - loss: 0.0595 - val_loss: 0.0225
Epoch 6/100
480/480 [==============================] - 79s 165ms/step - loss: 0.0499 - val_loss: 0.0015
Epoch 7/100
480/480 [==============================] - 78s 163ms/step - loss: 0.0461 - val_loss: 0.0033
Epoch 8/100
480/480 [==============================] - 76s 157ms/step - loss: 0.0370 - val_loss: 0.0124
Epoch 9/100
480/480 [==============================] - 75s 157ms/step - loss: 0.0322 - val_loss: 0.0012
Epoch 10/100
480/480 [==============================] - 79s 164m

KeyboardInterrupt: 